In [ ]:
import pandas as pd

# Extract genes based on statistical analysis
variant = 'statistical'
targetGenes = [] # TODO VINCE: Update this file to do statistical analysis (variation between this gene values, etc.) to come up with a list of genes to use.
# Don't forget to show your works (add more cells, print intermediate results)
df = pd.read_csv('../Data/clinical.csv')
geneFiles = df[['file', 'tnbc', 'bcr_patient_uuid']]

geneFiles

,file,tnbc,bcr_patient_uuid
0,84225715-14a6-423c-a6d6-15558e151f56\253aa5dc-...,False,6E7D5EC6-A469-467C-B748-237353C23416
1,927604f9-a38e-4c3f-b50f-3e0db4daf5ec\1be6a56c-...,False,55262FCB-1B01-4480-B322-36570430C917
2,931442ba-af81-4b68-beca-7285fc44b1df\f2dda955-...,False,427D0648-3F77-4FFC-B52C-89855426D647
3,7b4d770a-2b8c-4ca5-bf51-c4745c5de39a\ae8996bd-...,False,C31900A4-5DCD-4022-97AC-638E86E889E4
4,307261f2-f88f-4658-b6d1-98ef946148e2\75d91076-...,False,6623FC5E-00BE-4476-967A-CBD55F676EA6
...,...,...,...
972,0a7dc8b9-4196-41d3-ada1-a50cb36bfd2b\3c9c665e-...,False,5CD79093-1571-4F71-8136-0D84CCABDCAC
973,e0cb738d-854a-4033-b370-79fa28d7cef8\69118aea-...,False,F89588E9-CA73-4465-A7FB-7246EDB45E3A
974,e25bb12b-f88b-41c5-951e-a36d4b94008f\73e13f2d-...,False,CA20249F-B7EA-4FD9-9ECB-34F74755AE35
975,ee447251-5c90-426f-a0c7-bd2041189761\06a03e45-...,False,23F438BD-1DBB-4D46-972F-1E8E74DDBD37


In [2]:
dfPatientGenes = pd.DataFrame()
total_files = len(geneFiles)
for idx, row in geneFiles.iterrows():
    file_name = row.iloc[0]
    file_path = f"../Data/{file_name}"
    
    print(f"File {idx+1}/{total_files} - {file_name}", end="\r")
    
    dfGenes = pd.read_csv(file_path, sep='\t', skiprows=[0])
    
    # Columns may have been dropped if first line in file were columns, load data again without dropping first line
    if "gene_name" not in dfGenes.columns:
        dfGenes = pd.read_csv(file_path, sep='\t')
    
    #dfGenes
    
    # Gather all targeted genes we want with the values we want
    dfTarget = dfGenes[dfGenes['gene_name'].isin(targetGenes)][['gene_name', 'stranded_first']]

    # Transform the dfTarget to a single row dataframe with all genes as columns
    dfNewGenes = dfTarget.set_index('gene_name').T.reset_index(drop=True)

    # Add tnbc status
    dfNewGenes["tnbc"] = df["tnbc"].loc[df['file'] == file_name].values

    # Add the patient uuid for reference and set it as the index
    patientID = df['bcr_patient_uuid'].loc[df['file'] == file_name]
    dfNewGenes['case_id'] = patientID.values
    
    # Concat the data to the final dataframe before saving it to a file
    dfPatientGenes = pd.concat([dfPatientGenes, dfNewGenes])
    
#Send data to a csv
dfPatientGenes.to_csv(f'../Data/patient_genes_{variant}.csv', index=False)

dfPatientGenes

gene_name,CACNA2D2,ESR1,AGR2,GATA3,SLC16A6,TBC1D9,INPP4B,LDHB,MLPH,TSPAN1,...,STARD3,RARA,MCCC2,PSAT1,MFGE8,ANXA9,PPP1R14C,SLC44A4,tnbc,case_id
0,2934,5459,2397,22694,533,28741,1411,77,9991,7790,...,1735,5474,7298,27,1338,4083,7,2522,False,6E7D5EC6-A469-467C-B748-237353C23416
0,1146,83,31977,7581,2621,2484,696,33,18941,1045,...,9801,1262,4188,515,2026,255,134,2266,False,55262FCB-1B01-4480-B322-36570430C917
0,179,4387,4988,6374,217,3038,559,42,3081,839,...,1467,7566,3067,124,1390,545,0,616,False,427D0648-3F77-4FFC-B52C-89855426D647
0,163,3242,19966,11680,1658,8178,4037,89,14805,2945,...,5490,3562,7924,42,2808,1426,57,5938,False,C31900A4-5DCD-4022-97AC-638E86E889E4
0,410,6934,8357,17783,862,6620,3515,50,8179,798,...,1595,3981,9489,93,4476,380,48,2005,False,6623FC5E-00BE-4476-967A-CBD55F676EA6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,380,2912,652,4759,52,2108,194,65,25447,88,...,544,1756,1680,9,1058,449,8,4069,False,5CD79093-1571-4F71-8136-0D84CCABDCAC
0,407,3735,10314,10250,2380,10281,1711,94,12496,155,...,1624,2523,8242,104,3345,858,13,3064,False,F89588E9-CA73-4465-A7FB-7246EDB45E3A
0,740,2884,6575,7760,127,4557,746,122,8454,1219,...,1289,1516,3283,222,2996,439,5,6097,False,CA20249F-B7EA-4FD9-9ECB-34F74755AE35
0,257,2944,5463,4789,105,4347,543,290,5357,1956,...,1274,2046,1921,49,5857,362,216,1984,False,23F438BD-1DBB-4D46-972F-1E8E74DDBD37
